### Pong

Vamos a empezar a hacer un juego completo, empezaremos con un juego muy
sencillo e histórico: El **Pong**.

> Nota:Aunque existieron con anterioridad otros videojuegos de las dos décadas anteriores como [OXO](https://es.wikipedia.org/wiki/OXO) (ejecutado en una computadora única en el mundo) y posterior a éste [Spacewar!](https://es.wikipedia.org/wiki/Spacewar!) bajo la PDP-1 de DEC, eran, en su mayoría, proyectos experimentales: Pong está considerado por muchos como el más importante de entre la primera generación de videojuegos modernos, debido a que fue el primero en comercializarse a nivel masivo y no ejecutarse en máquinas únicas. Más información sobre el [Pong en la wikipedia](https://es.wikipedia.org/wiki/Pong)

In [7]:
from IPython.display import HTML

HTML('''
<iframe width="560" height="315" src="https://www.youtube.com/embed/e4VRgY3tkh0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>
''')

In [8]:
### Primeros pasos

Empezamos con los `imports` habituales:

In [2]:
import pygame
import random
import time

Y algunas constantes también habituales: El tamaño de la pantalla y algunos colores básicos:

In [4]:
# Tamaño de pantalla

SIZE = WIDTH, HEIGHT = 800, 640

# Colores

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

# Velocidad del juego
FPS = 30

Vamos a empezar programando la pelota, de forma parecida a como hicimos con las estrellas en
el ejercicio anterior. Aunque solo necesitamos una pelota, en la mayoría de los lenguajes orientados
a objetos se usa el concepto de clase, lo que significa que tenemos que escribir una clase pelota
y luego instanciar un objeto de tipo pelota:


In [43]:
class Ball:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def update(self):
        self.x += 1
    
    def draw(self, canvas):
        rect = pygame.Rect((self.x-5, self.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)
        

Nuestra clase `Ball` tiene dos atributos, `x` e `y`, para saber su posición en la pantalla,
y dos métodos, uno llamado `update` que servira para actualizar el estado de la 
pelota (Por el momento no hace nada). y otro llamada `draw` que dibuja la pelota, que en
nuestro caso es simplemente un cuadrado blanco.

El método `draw` utiliza una clase que no hemos usado antes, `Rect`. Resulta 
que los rectángulos son muy útiles a la hora de desarrollar un videojuego. Podemos
especificar un rectángulo usando una tupla de cuatro elementos, pero
pygame define su propia clase para rectángulos porque eso le permite 
añadir determinados atributos y métodos muy útiles, como por ejemplo, 
saber si dos rectángulos se solapan, modificar al ancho y el alto, detectar
si un punto está dentro o fuera del rectángulo, etc...

Para crear un rectángulo, podemos hacerlo de dos maneras:

1) La primera es pasándole cuatro parámetros: La posición x de la esquina superior derecha, la
posición y de la misma esquina, el ancho del rectangulo y el alto del rectangulo.

2) La segunda es pasarle dos parametros, cada uno de los cuales es una dupla (una tupla
de dos elementos). La primera dupla son las coordenadas x e y, y la segunda el ancho y el alto.

Es decir, las siguientes dos lineas crean dos rectángulos exactamente iguales:

In [41]:
r1 = pygame.Rect(50, 50, 100, 150)
r2 = pygame.Rect((50, 50), (100, 150))
assert r1 == r2
r1.center

(100, 125)

Según el caso puede ser mñas conveniente usar un sistema o el otro. 
Para el caso de la palota, dibujamos un rectángulo con la esquina superior izquierda deplazada
5 pixels a la derecha y hacia arriba, y luego indicamos un ancho y alto de 11 pixels, lo que
nos da un rectangulo de 11x11 pixels, donde la coordenadas x y y apuntan justo al centro del mismo.

In [42]:
pygame.init()
try:
    pygame.display.set_caption("Starfield")
    screen = pygame.display.set_mode(SIZE, 0, 24)

    # Parte de inicialización del juego
    ball = Ball(WIDTH//2, HEIGHT//2)
    clock = pygame.time.Clock()    
    in_game = True

    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        ball.update()
        # Representamos el nuevo estado
        screen.fill(BLACK)
        ball.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

Si ejecutamos el código anterior, vemos que la pelota está inmovil en el centro. 

**Ejercicio**: Hacer que la pelota se desplace. Lo único que hay que hacer
es tocar el método `update` de la clase Ball.

**Ejercicio**: La pelota va muy lenta. ¿Cómo podemos hacer para que vaya más rápida?

**Ejercicio**: La pelota se sale por los bordes de la pantalla. ¿Cómo podemos hace para 
que, en vez de desaparecer, aparezca por el borde contrario? ¿Y, si en vez de eso, queremos que rebote?

### Vectores y álgebra lineal

Si hemos resuelto el problema anterior, habremos usado dos valores para indicar la velocidad en el 
eje x y en el eje y. De la misma manera, hemos usado dos valores conbinados para indicar la
posicion de la pelota: la coordenada x y la y.
    
los matemáticos tienen un nombre para eestos casos en que combinamos dos o más números, se
llaman **vectores**. Y es uno de esos conceptos matemáticos que se usan muchísimo
en el desarrollo de juegos. El área de las matemáticas que estudia los vectores y otros
conceptos relacionados se llama [Álgebra lineal](https://es.wikipedia.org/wiki/%C3%81lgebra_lineal).

Los vectores se pueden usar, como en el caso anterio, para representar distintos
tipos de datos. Por ejemplo, podemos usar un vector para representar un punto o una
posicion en el espacio (En cuyo caso se habla normalmente de punto, para
hacer entender que estamos interesados sobre todo en una posición física). Pero
también se pueden usar para otras cosas, como por ejemplo, para indicar la velocidad
de un objeto, un cuyo caso es habitual hablar de vectores (vector de movimiento, vector
de acceleración, etc...



pygame no define su propia clase de vectores, pero es muy fácil crear nuestra propia 
clase. Vamos a crear una clase `Vector2` (El $2$ indica que en este caso
son vectores de dos dimensiones, más adelante definiremos una clase `Vector3` para 
trabajar en tres dimensiones, añadiendo una tercera coordenada $z$). Esta clase nos permite
referirnos a las coordenadas de forma cómoda con los atributos `x``e `y`:

In [71]:
class Vector2:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return 'Vector2({:.3f}, {:3f})'.format(self.x,self.y)
        
v = Vector2(3, 4)
assert v.x == 3
assert v.y == 4

Además dmel constructor, con su nombre _mágico_ especial, `__init__`, hemos definido un 
nuevo método `magico`, en este caso llamado `__str__`. Este mñétodo será llamada cada
vez que sea necesario convertir un objeto de la clase `Vector2` a una string, por ejemplo, 
para imprimirlo con la función `print`:

In [64]:
v = Vector2(3, 4)
print(v)

Vector2(3, 4)


**Ejercicio**: Comentar el método `__str__` y, luego de ejecutar la celda para que
se compile la nueva versión, llamar a print otra vez a ver que sale.

Una de las formas más habituales de crear un vector es a partir de dos puntos (es decir, a
partir de otros dos puntos o vectores). Por ejemplo, si tenemos la posición del personaje
de nuestro jugador, y tambien tenemos la posicion de un tesoro, puede que nos interes obtener el vector que apunta, partiendo del personaje, hacia el tesoro, como se ve en el siguiente diagrama:

![Mapa del tesoro](./art/mapa-del-tesoro.svg)

In [72]:
def vector_from_points(v1, v2):
    return Vector2( v2.x - v1.x, v2.y - v1.y)

In [73]:
v = vector_from_points(Vector2(1.2, 3.1), Vector2(3.6, 1.3))
print(v)

Vector2(2.400, -1.800000)


Este vector representa la distancia y la orientacion del movimiento que
tendría que hacer el jugador para llegar al tesoro.

## Módulo o magnitud de un vector

Un concepto importante asociado a los vectores es el de **magnitud** o **módulo** del vector, que
es simplemente la distancia entre dos puntos. En el caso anterior, si quisieramos
saber cuantos kilómetros (o pixels) tiene que moverse nuestro personaje, esa información
nos la da el módulo. 

Podemos definir nuestro propio método para poder calcular el módulo de
un vector, se trata solo de implementar la función de la [distancia euclidea](https://es.wikipedia.org/wiki/Distancia_euclidiana), es decir
la raiz cuadrada de la suma de los cuadrados de las coordenadas $x$ e $y$:

In [74]:
import math

class Vector2:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return 'Vector2({:.3f}, {:3f})'.format(self.x,self.y)
    
    def mod(self):
        return math.sqrt(self.x**2 + self.y**2)
        
v = Vector2(3, 4)
print(v.mod())

5.0


In [76]:
### Vectores unitarios

Los vectores definen dos cosas: una distancia (módulo o magnitudd) y una dirección. Normalmente
esta información están implicitas en el propio vector, pero en determinados casos podemos querer
trabajar con esos dos conceptos por separado. Para la magnitud o modulo solo necesitamos
un número, pero para indicar la direccion usaremos lo que se llama un **vector unitario** que
apunte en la misma direccion que el vector original. Un vector unitario es, simplemente, un vector cuyo módulo es $1$.

La operación de obtener un vector unitario a partir de un vector cualquiera se denomina **normalización**.

Podemos obtener un vector unitario a partir de un vector normal dividiendo cada una de las coordenadas del vector por su módulo. Implementemos un método `normalize` que haga exactamente esto:

In [79]:
import math

class Vector2:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return 'Vector2({:.3f}, {:3f})'.format(self.x,self.y)
    
    def mod(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    def normalize(self):
        d = self.mod()
        return Vector2(self.x / d, self.y / d)

In [81]:
v = Vector2(90, 90)
print(v.normalize())

Vector2(0.707, 0.707107)


Todo vector unitario tiene módulo $1$

In [84]:
v = Vector2(90, 90).normalize().mod() == 1

## Suma de vectores

La suma de vectores toma dos argumentos de tipo Vector y produce un nuevo
vector resultante, que viene a ser el resultado de combinar los dos. Por ejemplo,
si me quiero mover 5 pixels a la derecha y 4 pixel arriba (Vector2(5, -4)) y luego
me quiero mover 2 pixeles a la izquierda y 2 pixeles arriba (Vector2(-2, -2)), mis coordenadas
finales serán 3, -6, es cedir que:
    
    Vector2(5, -4) + Vector2(-2, -2)
    
    
Debería ser igual a:
    
    Vector2(3, -6)
    
Podemos hacer un método o una función `add` para sumar vectores:

In [87]:
def  add(v1, v2):
    return Vector2(v1.x + v2.x, v1.y + v2.y)

v1 = Vector2(5, -4)
v2 = Vector2(-2, -2)
v3 = add(v1, v2)
print(v3)

Vector2(3.000, -6.000000)


pero Python nos proporciona un sistema mucho mejor, llamado _sobrecarga de operadores_, por el cual podemos definir como se comportan nuestro objetos ante determinadas operaciones, como por ejemplo, la suma. Para ello, usa una serie de nombres mágicos (como `__init__` y `__str__`,ya vistos). En el
caso de la suma, debemos implemenar un ḿetodo que se debe llamar `__add__`.

Veamos como implementar esto en nuestra clase Vector2:

In [89]:
import math

class Vector2:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return 'Vector2({:.3f}, {:3f})'.format(self.x,self.y)
    
    def mod(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    def normalize(self):
        d = self.mod()
        return Vector2(self.x / d, self.y / d)
    
    def __add__(self, other):
        return Vector2(self.x+other.x, self.y+other.y)

Y ahora podemos sumar vectores de una forma mucho más legible y cómoda:

In [90]:
v1 = Vector2(5, -4)
v2 = Vector2(-2, -2)
v3 = v1 + v2
print(v3)

Vector2(3.000, -6.000000)
